# Graph2Text

In [6]:
import json
from pathlib import Path

def load_taxonomy(taxonomy_path: str):
    """Load taxonomy mapping from taxonomy.json."""
    t = json.loads(Path(taxonomy_path).read_text(encoding="utf-8"))
    return {int(k): v for k, v in t.get("id2room", {}).items()}

def articleize(label: str) -> str:
    """Add 'the', 'a', or 'an' before a label depending on plurality."""
    clean = label.strip().replace("_", " ")
    lower = clean.lower()

    # heuristic plural detection
    if lower.endswith(("s", "x", "z", "ch", "sh")) and not lower.endswith(("ss", "us")):
        article = "a"
    else:
        # singular
        vowels = "aeiou"
        article = "an" if lower[0] in vowels else "the"
    return f"{article} {clean}"

def graph2text(graph_path: str, taxonomy: dict, max_edges: int = 10_000):
    """
    Converts either a 3D-FRONT room graph or scene graph JSON to text.
    Uses taxonomy to decode room_id when available.
    Removes underscores and adds articles ('the', 'a', 'an').
    """
    path = Path(graph_path)
    g = json.loads(path.read_text(encoding="utf-8"))

    nodes = g.get("nodes", [])
    edges = g.get("edges", [])
    if not edges:
        return ""

    is_scene_graph = "room_a" in edges[0] or "room_b" in edges[0]

    # build node label map
    id_to_label = {}
    for n in nodes:
        if is_scene_graph:
            rid = n.get("room_id")
            raw_label = taxonomy.get(rid, n.get("room_type", str(rid)))
        else:
            raw_label = n.get("label", n.get("id"))
        id_to_label[n["id"]] = articleize(raw_label)

    sentences = []
    seen = set()

    for e in edges[:max_edges]:
        a = e.get("room_a") if is_scene_graph else e.get("obj_a")
        b = e.get("room_b") if is_scene_graph else e.get("obj_b")
        if not a or not b:
            continue

        label_a = id_to_label.get(a)
        label_b = id_to_label.get(b)
        if not label_a or not label_b:
            continue

        key = tuple(sorted([label_a, label_b]))
        if key in seen:
            continue
        seen.add(key)

        dist = e.get("distance_relation")
        direc = e.get("direction_relation")

        if dist and direc:
            sentence = f"{label_a} is {dist} and {direc} {label_b}."
        elif dist:
            sentence = f"{label_a} is {dist} {label_b}."
        elif direc:
            sentence = f"{label_a} is {direc} {label_b}."
        else:
            sentence = f"{label_a} relates to {label_b}."

        sentences.append(sentence)

    text = " ".join(sentences)
    return text.replace("_", " ")


# --- Example ---
taxonomy = load_taxonomy(r"C:\Users\Hagai.LAPTOP-QAG9263N\Desktop\Thesis\repositories\ImagiNav\config\taxonomy.json")
# --- Example usage ---
scene_graph_path = r"C:\Users\Hagai.LAPTOP-QAG9263N\Desktop\Thesis\repositories\ImagiNav\dataset\00b88e19-d106-4ab8-a322-31c494a0a6b9\00b88e19-d106-4ab8-a322-31c494a0a6b9_scene_graph.json"
room_graph_path = r"C:\Users\Hagai.LAPTOP-QAG9263N\Desktop\Thesis\repositories\ImagiNav\dataset\00b88e19-d106-4ab8-a322-31c494a0a6b9\rooms\3020\layouts\00b88e19-d106-4ab8-a322-31c494a0a6b9_3020_graph.json"  # or any *_graph.json
scene_description = graph2text(scene_graph_path,taxonomy)
room_description = graph2text(room_graph_path,taxonomy)
print(room_description)  # preview first 1000 chars


the Cabinet/Shelf/Desk is near and front of the Cabinet/Shelf/Desk. the Cabinet/Shelf/Desk is left of the Chair. the Cabinet/Shelf/Desk is left of the Lighting. the Cabinet/Shelf/Desk is behind the Table. the Cabinet/Shelf/Desk is behind the Structure. the Cabinet/Shelf/Desk is left of the Pier/Stool. the Chair is near and front of the Lighting. the Chair is left of the Table. the Chair is left of the Structure. the Chair is near and front of the Pier/Stool. the Lighting is left of the Table. the Lighting is left of the Structure. the Lighting is near and front of the Pier/Stool. the Table is behind the Table. the Table is by and front of the Structure. the Table is front of the Pier/Stool. the Structure is left of the Structure. the Structure is right of the Pier/Stool.


# Graph Embeddings

In [4]:
from sentence_transformers import SentenceTransformer
import numpy as np

# load pretrained model (first download may take ~100 MB)
embedder = SentenceTransformer("all-MiniLM-L6-v2")

# assume you already ran:
# text = graph2text_unified(path, taxonomy)
embedding = embedder.encode(room_description, normalize_embeddings=True)

print("Embedding shape:", embedding.shape)
print("First 10 values:", np.round(embedding[:10], 4))


Embedding shape: (384,)
First 10 values: [ 0.062   0.0255  0.0034  0.007   0.0384  0.0523 -0.0622 -0.0122  0.0583
  0.0503]


c:\Users\Hagai.LAPTOP-QAG9263N\.conda\envs\ImgiNav\lib\site-packages\torch\nn\modules\module.py:1527: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [7]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer("all-MiniLM-L6-v2")

graph_text = "the Cabinet/Shelf/Desk is near and front of the Cabinet/Shelf/Desk. the Cabinet/Shelf/Desk is left of the Chair. the Cabinet/Shelf/Desk is left of the Lighting. the Cabinet/Shelf/Desk is behind the Table. the Cabinet/Shelf/Desk is behind the Structure. the Cabinet/Shelf/Desk is left of the Pier/Stool. the Chair is near and front of the Lighting. the Chair is left of the Table. the Chair is left of the Structure. the Chair is near and front of the Pier/Stool. the Lighting is left of the Table. the Lighting is left of the Structure. the Lighting is near and front of the Pier/Stool. the Table is behind the Table. the Table is by and front of the Structure. the Table is front of the Pier/Stool. the Structure is left of the Structure. the Structure is right of the Pier/Stool."
free_text  = "A room with a desk positioned near the front, to the left of a chair and a lamp. The lamp and chair stand close together in front of a small table and a structural element like a wall or pillar. A stool sits toward the right side of the structure."

emb_graph = model.encode(graph_text, normalize_embeddings=True)
emb_free  = model.encode(free_text, normalize_embeddings=True)

similarity = util.cos_sim(emb_graph, emb_free).item()
print(f"Cosine similarity: {similarity:.3f}")


Cosine similarity: 0.695
